In [196]:
import pandas as pd
import numpy as np
import tmdbsimple as tds
import time
import pdb
import ast
keys = pd.read_csv('keys.txt')
keys['kDb3'][0]
tds.API_KEY = keys['kDb3'][0]

import requests # Not yet used
from IPython.display import Image # Not yet used
from IPython.core.display import HTML # Not yet used

import datetime
#from datetime import date # Used for date.today() to find age of actor who is still alive.

In [2]:
#Settings
# Set ipython's max row display
pd.set_option('display.max_row', 50)

# Set iPython's max column width to 50
pd.set_option('display.max_columns', 50)

In [14]:
%pdb

Automatic pdb calling has been turned OFF


# IMDb Source

In [3]:
print('Parsing Title AKAs')
tadf = pd.read_csv('https://datasets.imdbws.com/title.akas.tsv.gz', sep='\t')
print('Parsing Title Basics')
tbdf = pd.read_csv('https://datasets.imdbws.com/title.basics.tsv.gz', sep='\t')
print('Parsing Title Crews')
tcdf = pd.read_csv('https://datasets.imdbws.com/title.crew.tsv.gz', sep='\t')
print('Parsing Title Principals')
tpdf = pd.read_csv('https://datasets.imdbws.com/title.principals.tsv.gz', sep='\t')
print('Parsing Title Episodes')
tedf = pd.read_csv('https://datasets.imdbws.com/title.episode.tsv.gz', sep='\t')
print('Parsing Title Ratings')
trdf = pd.read_csv('https://datasets.imdbws.com/title.ratings.tsv.gz', sep='\t')
print('Parsing Name Basics')
nbdf = pd.read_csv('https://datasets.imdbws.com/name.basics.tsv.gz', sep='\t')

Parsing Title AKAs


/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Parsing Title Basics


/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Parsing Title Crews
Parsing Title Principals
Parsing Title Episodes
Parsing Title Ratings
Parsing Name Basics


https://www.imdb.com/interfaces/

# IMDb Dataset Details

Each dataset is contained in a gzipped, tab-separated-values (TSV) formatted file in the UTF-8 character set. The first line in each file contains headers that describe what is in each column. A ‘\N’ is used to denote that a particular field is missing or null for that title/name. The available datasets are as follows:

## title.akas.tsv.gz
Contains the following information for titles: titleId (string) - a tconst, an alphanumeric unique identifier of the title ordering (integer) – a number to uniquely identify rows for a given titleId title (string) – the localized title region (string) - the region for this version of the title language (string) - the language of the title types (array) - Enumerated set of attributes for this alternative title. One or more of the following: "alternative", "dvd", "festival", "tv", "video", "working", "original", "imdbDisplay". New values may be added in the future without warning attributes (array) - Additional terms to describe this alternative title, not enumerated isOriginalTitle (boolean) – 0: not original title; 1: original title

## title.basics.tsv.gz
Contains the following information for titles: tconst (string) - alphanumeric unique identifier of the title titleType (string) – the type/format of the title (e.g. movie, short, tvseries, tvepisode, video, etc) primaryTitle (string) – the more popular title / the title used by the filmmakers on promotional materials at the point of release originalTitle (string) - original title, in the original language isAdult (boolean) - 0: non-adult title; 1: adult title startYear (YYYY) – represents the release year of a title. In the case of TV Series, it is the series start year endYear (YYYY) – TV Series end year. ‘\N’ for all other title types runtimeMinutes – primary runtime of the title, in minutes genres (string array) – includes up to three genres associated with the title

## title.crew.tsv.gz
Contains the director and writer information for all the titles in IMDb. Fields include: tconst (string) - alphanumeric unique identifier of the title directors (array of nconsts) - director(s) of the given title writers (array of nconsts) – writer(s) of the given title

## title.episode.tsv.gz
Contains the tv episode information. Fields include: tconst (string) - alphanumeric identifier of episode parentTconst (string) - alphanumeric identifier of the parent TV Series seasonNumber (integer) – season number the episode belongs to episodeNumber (integer) – episode number of the tconst in the TV series

## title.principals.tsv.gz
Contains the principal (highest ranking, important) cast/crew for titles tconst (string) - alphanumeric unique identifier of the title ordering (integer) – a number to uniquely identify rows for a given titleId nconst (string) - alphanumeric unique identifier of the name/person category (string) - the category of job that person was in job (string) - the specific job title if applicable, else '\N' characters (string) - the name of the character played if applicable, else '\N'

## title.ratings.tsv.gz
Contains the IMDb rating and votes information for titles tconst (string) - alphanumeric unique identifier of the title averageRating – weighted average of all the individual user ratings numVotes - number of votes the title has received

## name.basics.tsv.gz
Contains the following information for names: nconst (string) - alphanumeric unique identifier of the name/person primaryName (string)– name by which the person is most often credited birthYear – in YYYY format deathYear – in YYYY format if applicable, else '\N' primaryProfession (array of strings)– the top-3 professions of the person knownForTitles (array of tconsts) – titles the person is known for


### Data Uses
tbdf can be used to filter types and genres

tcdf can be used to credit directors and writers of a film

nbdf can be used to find the names of the aforementioned directors and writers, as well as actors and their known production work titles.

trdf can be used to target the actor's

tadf can be used to target a film's primary country of origin for a general idea of location of production relevance

In [4]:
print('Title AKAs')
print(tadf.info())
print('\n')
print('Title Basics')
print(tbdf.info())
print('\n')
print('Title Crews')
print(tcdf.info())
print('\n')
print('Title Principals')
print(tpdf.info())
print('\n')
print('Title Episodes')
print(tedf.info())
print('\n')
print('Title Ratings')
print(trdf.info())
print('\n')
print('Name Basics')
print(nbdf.info())
print('\n')

Title AKAs
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25778868 entries, 0 to 25778867
Data columns (total 8 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   titleId          object
 1   ordering         int64 
 2   title            object
 3   region           object
 4   language         object
 5   types            object
 6   attributes       object
 7   isOriginalTitle  object
dtypes: int64(1), object(7)
memory usage: 1.5+ GB
None


Title Basics
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7770807 entries, 0 to 7770806
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 533.6+ MB
None


Title Crews
<class 'pandas.core.frame.DataFram

# TMDb Source

In [21]:
# https://docs.python.org/3/library/datetime.html#strftime-strptime-behavior
datetime.date.strftime(datetime.date.today(), '%m_%d_%Y')

'04_02_2021'

In [22]:
retrieval_date = datetime.date.today()- datetime.timedelta(days=1)

In [23]:
print('Parsing Movies')
tmmdf = pd.read_json(f"http://files.tmdb.org/p/exports/movie_ids_{datetime.date.strftime(retrieval_date, '%m_%d_%Y')}.json.gz", lines = True)
print(tmmdf.info())
print('Parsing TV Series')
tmtsdf = pd.read_json(f"http://files.tmdb.org/p/exports/tv_series_ids_{datetime.date.strftime(retrieval_date, '%m_%d_%Y')}.json.gz", lines = True)
# skipping for this analysis
print(tmtsdf.info())
print('Parsing People')
tmpdf = pd.read_json(f"http://files.tmdb.org/p/exports/person_ids_{datetime.date.strftime(retrieval_date, '%m_%d_%Y')}.json.gz", lines = True)
print(tmpdf.info())
print('Parsing Collections')
tmcdf = pd.read_json(f"http://files.tmdb.org/p/exports/collection_ids_{datetime.date.strftime(retrieval_date, '%m_%d_%Y')}.json.gz", lines = True)
print(tmcdf.info())
print('Parsing TV Networks')
tmtndf = pd.read_json(f"http://files.tmdb.org/p/exports/tv_network_ids_{datetime.date.strftime(retrieval_date, '%m_%d_%Y')}.json.gz", lines = True)
print(tmtndf.info())
print('Parsing Keywords')
tmkdf = pd.read_json(f"http://files.tmdb.org/p/exports/keyword_ids_{datetime.date.strftime(retrieval_date, '%m_%d_%Y')}.json.gz", lines = True)
print(tmkdf.info())
print('Parsing Production Companies')
tmpcdf = pd.read_json(f"http://files.tmdb.org/p/exports/production_company_ids_{datetime.date.strftime(retrieval_date, '%m_%d_%Y')}.json.gz", lines = True)
print(tmpcdf.info())

Parsing Movies
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 594176 entries, 0 to 594175
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   adult           594176 non-null  bool   
 1   id              594176 non-null  int64  
 2   original_title  594176 non-null  object 
 3   popularity      594176 non-null  float64
 4   video           594176 non-null  bool   
dtypes: bool(2), float64(1), int64(1), object(1)
memory usage: 14.7+ MB
None
Parsing TV Series
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106797 entries, 0 to 106796
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   id             106797 non-null  int64  
 1   original_name  106797 non-null  object 
 2   popularity     106797 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 2.4+ MB
None
Parsing People
<class 'pandas.core.frame.DataFrame'>
Ran

In [153]:
sorted_tmdids = list(tmmdf['id'].sort_values())#.set_index('id')

In [159]:
sorted_tmdids

[2,
 3,
 5,
 6,
 8,
 9,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 24,
 25,
 26,
 27,
 28,
 30,
 31,
 32,
 33,
 35,
 38,
 55,
 58,
 59,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 182,
 183,
 184,
 185,
 186,
 187,
 189,
 190,
 191,
 192,
 193,
 194,
 195,
 196,
 197,
 198,
 199,
 200,
 201,
 203,
 204,
 205,
 206,
 207,
 211,
 212,
 213,
 214,
 215,
 216,
 217,
 218,
 219,
 220,


# TMDb Data Pull Commentary

In [51]:
tds.Find('tt0133093').info(external_source='imdb_id') # Can use IMDB to search TMDB
# This search gives brief info and the TMDB ID to use for further info.

{'movie_results': [{'adult': False,
   'backdrop_path': '/fNG7i7RqMErkcqhohV2a6cV1Ehy.jpg',
   'genre_ids': [28, 878],
   'id': 603,
   'original_language': 'en',
   'original_title': 'The Matrix',
   'poster_path': '/f89U3ADr1oiB1s9GkdPOEpXUk5H.jpg',
   'video': False,
   'title': 'The Matrix',
   'vote_count': 18912,
   'overview': 'Set in the 22nd century, The Matrix tells the story of a computer hacker who joins a group of underground insurgents fighting the vast and powerful computers who now rule the earth.',
   'release_date': '1999-03-30',
   'vote_average': 8.1,
   'popularity': 51.948}],
 'person_results': [],
 'tv_results': [],
 'tv_episode_results': [],
 'tv_season_results': []}

In [7]:
tds.Movies(603).info()

{'adult': False,
 'backdrop_path': '/fNG7i7RqMErkcqhohV2a6cV1Ehy.jpg',
 'belongs_to_collection': {'id': 2344,
  'name': 'The Matrix Collection',
  'poster_path': '/lh4aGpd3U9rm9B8Oqr6CUgQLtZL.jpg',
  'backdrop_path': '/bRm2DEgUiYciDw3myHuYFInD7la.jpg'},
 'budget': 63000000,
 'genres': [{'id': 28, 'name': 'Action'},
  {'id': 878, 'name': 'Science Fiction'}],
 'homepage': 'http://www.warnerbros.com/matrix',
 'id': 603,
 'imdb_id': 'tt0133093',
 'original_language': 'en',
 'original_title': 'The Matrix',
 'overview': 'Set in the 22nd century, The Matrix tells the story of a computer hacker who joins a group of underground insurgents fighting the vast and powerful computers who now rule the earth.',
 'popularity': 52.143,
 'poster_path': '/f89U3ADr1oiB1s9GkdPOEpXUk5H.jpg',
 'production_companies': [{'id': 79,
   'logo_path': '/tpFpsqbleCzEE2p5EgvUq6ozfCA.png',
   'name': 'Village Roadshow Pictures',
   'origin_country': 'US'},
  {'id': 372,
   'logo_path': None,
   'name': 'Groucho II Film

# TMDB API Exploration and Uses

### alternative_titles
International names

### credits
Gives cast information. gender 1 = female, 2 = male. Gives name (guild) and original name (birth name). Gives credit ID and order name, which can be mildly useful.

##### known_for_department
A category in credits, offers best vals: Acting, Writing, Directing, Production

### keywords
Enables sub-genre analysis. Genres are from general .info() call. Useful for analysis.



In [9]:
# tds.Movies(603).alternative_titles() # International names.

# tds.Movies(603).credits() # Gives cast information. gender 1 = female, 2 = male. Gives name (guild)

# known_for_department best vals: Acting, Writing, Directing, Production
# Production = producers, Editing = Editors, Writing = writers, Directing = directors

# tds.Movies(603).keywords() # This enables sub-genre analysis

#.lists() # User-created lists with this item. Improper popularity measure.
# tds.Movies(603).recommendations() # TMBD's recommender system.

# tds.Movies(603).release_dates() # Release dates by county.

# tds.Movies(603).reviews() # Could be useful but users don't leave enough reviews on tmdb to be meaningful

{'id': 603,
 'keywords': [{'id': 83, 'name': 'saving the world'},
  {'id': 310, 'name': 'artificial intelligence'},
  {'id': 312, 'name': 'man vs machine'},
  {'id': 490, 'name': 'philosophy'},
  {'id': 530, 'name': 'prophecy'},
  {'id': 779, 'name': 'martial arts'},
  {'id': 1430, 'name': 'self sacrifice'},
  {'id': 1721, 'name': 'fight'},
  {'id': 2157, 'name': 'hacker'},
  {'id': 3074, 'name': 'insurgence'},
  {'id': 3972, 'name': 'simulated reality '},
  {'id': 4563, 'name': 'virtual reality'},
  {'id': 4565, 'name': 'dystopia'},
  {'id': 6256, 'name': 'truth'},
  {'id': 12190, 'name': 'cyberpunk'},
  {'id': 186789, 'name': 'dream world'},
  {'id': 187056, 'name': 'woman director'},
  {'id': 194063, 'name': 'messiah'},
  {'id': 219404, 'name': 'action hero'},
  {'id': 221385, 'name': 'gnosticism'}]}

In [175]:
tds.Movies(603).credits()

{'id': 603,
 'cast': [{'adult': False,
   'gender': 2,
   'id': 6384,
   'known_for_department': 'Acting',
   'name': 'Keanu Reeves',
   'original_name': 'Keanu Reeves',
   'popularity': 20.503,
   'profile_path': '/rRdru6REr9i3WIHv2mntpcgxnoY.jpg',
   'cast_id': 34,
   'character': 'Thomas A. Anderson / Neo',
   'credit_id': '52fe425bc3a36847f80181c1',
   'order': 0},
  {'adult': False,
   'gender': 2,
   'id': 2975,
   'known_for_department': 'Acting',
   'name': 'Laurence Fishburne',
   'original_name': 'Laurence Fishburne',
   'popularity': 5.512,
   'profile_path': '/8suOhUmPbfKqDQ17jQ1Gy0mI3P4.jpg',
   'cast_id': 21,
   'character': 'Morpheus',
   'credit_id': '52fe425bc3a36847f801818d',
   'order': 1},
  {'adult': False,
   'gender': 1,
   'id': 530,
   'known_for_department': 'Acting',
   'name': 'Carrie-Anne Moss',
   'original_name': 'Carrie-Anne Moss',
   'popularity': 6.362,
   'profile_path': '/ArlQHNKGUOlSDUHdIdjG6nLiT2A.jpg',
   'cast_id': 22,
   'character': 'Trinity',


In [179]:
# This retrieves images for image analysis
# Image(url= "https://www.themoviedb.org/t/p/w1280/tnAuB8q5vv7Ax9UAEje5Xi4BXik.jpg") 

# TMDB Attributes
https://developers.themoviedb.org/3/movies/get-movie-details

In [152]:
tcdf[tcdf['tconst'] == 'tt0133093']['writers'].values[0].split(',')[0]

'nm0905152'

In [162]:
str(date.today()).split('-')[0] # Prints the current year for use in calculating actor age.

'2021'

In [163]:
tds.Movies(603).info()

{'adult': False,
 'backdrop_path': '/fNG7i7RqMErkcqhohV2a6cV1Ehy.jpg',
 'belongs_to_collection': {'id': 2344,
  'name': 'The Matrix Collection',
  'poster_path': '/lh4aGpd3U9rm9B8Oqr6CUgQLtZL.jpg',
  'backdrop_path': '/bRm2DEgUiYciDw3myHuYFInD7la.jpg'},
 'budget': 63000000,
 'genres': [{'id': 28, 'name': 'Action'},
  {'id': 878, 'name': 'Science Fiction'}],
 'homepage': 'http://www.warnerbros.com/matrix',
 'id': 603,
 'imdb_id': 'tt0133093',
 'original_language': 'en',
 'original_title': 'The Matrix',
 'overview': 'Set in the 22nd century, The Matrix tells the story of a computer hacker who joins a group of underground insurgents fighting the vast and powerful computers who now rule the earth.',
 'popularity': 60.163,
 'poster_path': '/f89U3ADr1oiB1s9GkdPOEpXUk5H.jpg',
 'production_companies': [{'id': 79,
   'logo_path': '/tpFpsqbleCzEE2p5EgvUq6ozfCA.png',
   'name': 'Village Roadshow Pictures',
   'origin_country': 'US'},
  {'id': 372,
   'logo_path': None,
   'name': 'Groucho II Film

In [61]:
#tadf
#tbdf[tbdf['tconst']=='tt0457430']
#tcdf
#tpdf
#tedf
trdf
#nbdf

,tconst,averageRating,numVotes
0,tt0000001,5.6,1694
1,tt0000002,6.0,210
2,tt0000003,6.5,1441
3,tt0000004,6.1,122
4,tt0000005,6.1,2240
...,...,...,...
1135108,tt9916580,7.2,5
1135109,tt9916690,6.6,5
1135110,tt9916720,6.4,76
1135111,tt9916766,6.9,16


In [241]:
# Investigating how to use nameIDs efficiently.
nbdf.loc[nbdf['knownForTitles'].str.contains("tt0457430", case=False)] # this works!

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
6538,nm0006564,Víctor Albarrán,\N,\N,"production_manager,producer,assistant_director","tt0276437,tt0457430,tt0082198,tt0101025"
7566,nm0007614,Reyes Abades,1949,2018,"special_effects,actor,miscellaneous","tt4326444,tt0125659,tt1189073,tt0457430"
7567,nm0007615,Óscar Abades,\N,\N,"special_effects,art_department,miscellaneous","tt1189073,tt4326444,tt0457430,tt0913425"
16331,nm0016864,Fernando Albizu,1963,\N,actor,"tt0457430,tt0359045,tt1166810,tt6584918"
21323,nm0022136,Ángel Alonso,\N,\N,"special_effects,visual_effects,miscellaneous","tt0203119,tt0457430,tt0947810,tt4326444"
...,...,...,...,...,...,...
7673970,nm6163402,Álvaro Hernández Arriero,\N,\N,stunts,"tt1572491,tt0395119,tt0457430,tt0374569"
7673972,nm6163404,David Jiménez Cambón,\N,\N,stunts,"tt0457430,tt0375342"
7703278,nm6198885,Laura Bernice Watson,\N,\N,make_up_department,"tt5742374,tt2076298,tt0142688,tt0457430"
8024787,nm6602582,Chema Ruiz,\N,\N,music_department,tt0457430


# Data Collection Schema

1) Use tbdf to find all tconst from only movies. Take all columns from this table.

2) Use tadf to pull only movie data only from region=US. tbdf and tadf combine into tabdf

4) Use titleId to retrieve nameIds of directors and writers from tcdf. Merge into tabdf

5) Use titleId to find the principal crew of a film from tpdf. Note that when searching a titleId multiple rows appear, one per person nameId. Gender-specific "actor/actress" role names are used which can help for gender-specific identification.

6) SKIPPING tedf for this analysis. It is indexed by episode tt-id, with parent tt-id. Season and episode number columns are given.

7) Use titleId to find average rating and number of votes. This IMDB rating is distinct from TMDB ratings. IMDB seems to be more popular and thus has more ratings and reviews, but comparison between these and TMDB's ratings can be done.

8) Use nameIDs found from tcdf and tpdf on nbdf to locate names, birth/death years as primary data elements. knownForTitles can be occasionally useful. primaryProfession is granular, not sure if I would use this during this current analysis.

9) Notes on tc, tp, and nb dfs: use film tconst to start with tc for directors and writers. Although tp can make duplicate references to director and actor, tp does not indicate instances in which the director and writer are the same person, listing only director and no actor. Make a flag to indicate if the director and writer are the same person. Can use tconst to search for actors in nbdf that are known for that movie, which can link to other movies connected to those respective actors.

10) Use TMDb to find budget, keywords, popularity, revenue, and overview.

11) spoken_languages is new here but could be guessed from IMDB first via region.

12) genre, release_date, runtime, vote_average, and vote_count are found in IMDB but can be cross-referenced here.

In [252]:
tadf[tadf['region']=='US']['types'].unique()

array(['\\N', 'alternative', 'imdbDisplay', 'dvd', 'working', 'tv',
       'video', 'festival', 'original', 'tv\x02working', 'tv\x02video',
       'video\x02working', 'festival\x02working', 'dvd\x02video',
       'dvd\x02working'], dtype=object)

In [394]:
tadf[(tadf['region']=='US') & ((tadf['types'] == '\\N') | (tadf['types'] == 'imdbDisplay') )]

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,\N,\N,\N,0
14,tt0000002,7,The Clown and His Dogs,US,\N,\N,literal English title,0
40,tt0000005,6,Blacksmithing,US,\N,\N,informal alternative title,0
43,tt0000005,9,Blacksmith Scene,US,\N,\N,\N,0
46,tt0000006,3,Chinese Opium Den,US,\N,\N,\N,0
...,...,...,...,...,...,...,...,...
25767741,tt9916702,1,Loving London: The Playground,US,\N,\N,\N,0
25767780,tt9916720,12,The Nun 2,US,\N,imdbDisplay,\N,0
25767794,tt9916734,1,Manca: Peleo,US,\N,\N,\N,0
25767796,tt9916756,1,Pretty Pretty Black Girl,US,\N,\N,\N,0


In [395]:
tadf[(tadf['region']=='US') & (tadf['title'] == 'The Matrix')]

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
924621,tt0133093,27,The Matrix,US,\N,imdbDisplay,\N,0
5325993,tt11089880,1,The Matrix,US,\N,\N,\N,0
25251315,tt9642498,1,The Matrix,US,\N,imdbDisplay,\N,0


In [6]:
tadf['types'].unique()

array(['imdbDisplay', '\\N', 'original', 'alternative', 'dvd', 'festival',
       'working', 'tv', 'video', 'imdbDisplay\x02tv', 'alternative\x02tv',
       'imdbDisplay\x02working', 'festival\x02imdbDisplay',
       'tv\x02working', 'imdbDisplay\x02video', 'alternative\x02dvd',
       'tv\x02video', 'dvd\x02imdbDisplay', 'alternative\x02working',
       'video\x02working', 'alternative\x02video', 'festival\x02working',
       'dvd\x02video', 'alternative\x02festival', 'dvd\x02working'],
      dtype=object)

In [266]:
tbdf['titleType'].unique()

array(['short', 'movie', 'tvShort', 'tvMovie', 'tvSeries', 'tvEpisode',
       'tvMiniSeries', 'tvSpecial', 'video', 'videoGame', 'audiobook',
       'radioSeries', 'episode'], dtype=object)

In [9]:
tadf.rename(columns={'titleId':'tconst'}, inplace = True)

In [10]:
tadf[tadf['tconst'] == 'tt0000001']

,tconst,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0
5,tt0000001,6,Carmencita,US,\N,\N,\N,0
6,tt0000001,7,Carmencita,\N,\N,original,\N,1
7,tt0000001,8,カルメンチータ,JP,ja,imdbDisplay,\N,0


In [397]:
tadf['types'].unique()

array(['imdbDisplay', '\\N', 'original', 'alternative', 'dvd', 'festival',
       'working', 'tv', 'video', 'imdbDisplay\x02tv', 'alternative\x02tv',
       'imdbDisplay\x02working', 'festival\x02imdbDisplay',
       'tv\x02working', 'imdbDisplay\x02video', 'alternative\x02dvd',
       'tv\x02video', 'dvd\x02imdbDisplay', 'alternative\x02working',
       'video\x02working', 'alternative\x02video', 'festival\x02working',
       'dvd\x02video', 'alternative\x02festival', 'dvd\x02working'],
      dtype=object)

# Successful Merge of TA and TB DFs!

In [11]:
tabdf = tbdf[tbdf['titleType']=='movie'].merge(tadf[tadf['region']=='US'], on='tconst', how='inner')

In [12]:
tmovus = tabdf.merge(tcdf, on='tconst', how='inner') #was previously how='left'

In [13]:
tmovus = tmovus.merge(trdf, on='tconst', how='inner') #was previously how='left'

In [14]:
tmovus = tmovus.replace(to_replace=['\\N'], value=[None])
tmovus = tmovus.where(tmovus.notnull(), None)

In [15]:
tmovus

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,ordering,title,region,language,types,attributes,isOriginalTitle,directors,writers,averageRating,numVotes
0,tt0000630,movie,Hamlet,Amleto,0,1908,None,None,Drama,4,Hamlet,US,None,None,None,0,nm0143333,nm0000636,3.8,12
1,tt0000679,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0,1908,None,120,"Adventure,Fantasy",1,The Fairylogue and Radio-Plays,US,None,None,None,0,"nm0877783,nm0091767","nm0000875,nm0877783",5.2,35
2,tt0000886,movie,"Hamlet, Prince of Denmark",Hamlet,0,1910,None,None,Drama,2,"Hamlet, Prince of Denmark",US,None,None,None,0,nm0099901,nm0000636,5.0,24
3,tt0001101,movie,Abraham Lincoln's Clemency,Abraham Lincoln's Clemency,0,1910,None,None,None,2,Abraham Lincoln's Clemency,US,None,None,None,0,nm0923603,None,5.2,13
4,tt0001159,movie,The Connecticut Yankee,The Connecticut Yankee,0,1910,None,None,None,1,The Connecticut Yankee,US,None,None,None,0,None,None,5.6,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142915,tt9913660,movie,No Apology,No Apology,0,2019,None,102,Drama,4,No Apology,US,None,imdbDisplay,None,0,nm1067844,nm3014403,5.4,41
142916,tt9914642,movie,Albatross,Albatross,0,2017,None,97,Documentary,1,Albatross,US,None,None,None,0,nm5300859,"nm5300859,nm7332227",8.2,28
142917,tt9914644,movie,9/11: Escape from the Towers,9/11: Escape from the Towers,0,2018,None,120,Documentary,2,9/11: Escape from the Towers,US,None,imdbDisplay,None,0,nm10537376,None,8.3,67
142918,tt9916190,movie,Safeguard,Safeguard,0,2020,None,90,"Action,Adventure,Thriller",5,Safeguard,US,None,imdbDisplay,None,0,nm7308376,nm7308376,3.4,140


In [16]:
duplicate_movies = tmovus['tconst'].value_counts()
duplicate_movies[duplicate_movies > 1].shape

(19633,)

In [406]:
duplicate_movies[duplicate_movies > 3].shape

(1618,)

# This step is expensive! I estimate O^3, with the fully built algorithm taking O^5
## Only run the next cell if this data is needed!

In [ ]:
# Slow algorithm to get certain person names from IMDB.
# To be expanded in the future to calculate gender and age.

# tmovus['drnames'] = tmovus['directors'].apply(findnbdfn)
# tmovus['wrnames'] = tmovus['writers'].apply(findnbdfn)

In [17]:
tmovus.rename(columns={'genres':'genres_im', 'title': 'title_im'}, inplace = True)

In [18]:
tmovus[tmovus['tconst']=='tt0133093']

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres_im,ordering,title_im,region,language,types,attributes,isOriginalTitle,directors,writers,averageRating,numVotes
56366,tt0133093,movie,The Matrix,The Matrix,0,1999,None,136,"Action,Sci-Fi",27,The Matrix,US,None,imdbDisplay,None,0,"nm0905152,nm0905154","nm0905152,nm0905154",8.7,1695236


https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.astype.html

In [19]:
im_to_tm_sl = ['adult',
 'backdrop_path',
 'belongs_to_collection',
 'budget',
 'genres',
 'homepage',
 'id',
 'imdb_id',
 'original_language',
 'original_title',
 'overview',
 'popularity',
 'poster_path',
 'production_companies',
 'production_countries',
 'release_date',
 'revenue',
 'runtime',
 'spoken_languages',
 'status',
 'tagline',
 'title',
 'video',
 'vote_average',
 'vote_count']

In [205]:
im_to_tm_sl = list()
dfdts = dict()
for k, v in tds.Movies(603).info().items():
    im_to_tm_sl.append(k)
    dfdts[k] = type(v).__name__

In [209]:
dfdts

{'adult': 'bool',
 'backdrop_path': 'str',
 'belongs_to_collection': 'dict',
 'budget': 'int',
 'genres': 'list',
 'homepage': 'str',
 'id': 'int',
 'imdb_id': 'str',
 'original_language': 'str',
 'original_title': 'str',
 'overview': 'str',
 'popularity': 'float',
 'poster_path': 'str',
 'production_companies': 'list',
 'production_countries': 'list',
 'release_date': 'str',
 'revenue': 'int',
 'runtime': 'int',
 'spoken_languages': 'list',
 'status': 'str',
 'tagline': 'str',
 'title': 'str',
 'video': 'bool',
 'vote_average': 'float',
 'vote_count': 'int'}

In [ ]:
{'adult': 'bool',
 'backdrop_path': 'str',
 'belongs_to_collection': 'dict',
 'budget': 'int',
 'genres': 'list',
 'homepage': 'str',
 'id': 'int',
 'imdb_id': 'str',
 'original_language': 'str',
 'original_title': 'str',
 'overview': 'str',
 'popularity': 'float',
 'poster_path': 'str',
 'production_companies': 'list',
 'production_countries': 'list',
 'release_date': 'str',
 'revenue': 'int',
 'runtime': 'int',
 'spoken_languages': 'list',
 'status': 'str',
 'tagline': 'str',
 'title': 'str',
 'video': 'bool',
 'vote_average': 'float',
 'vote_count': 'int'}

In [24]:
# im_to_tm_collector global scope safety variables
tmdtdil = list() # TMDb return list
tmdedct = dict() # Error count dictionary
tmdrcid = list() # recent id before failure; to be assigned a string
tmdred = dict() # repeat ids and count of repeats 

In [25]:
def g_reset_tmd():
    tmdtdil.clear()
    tmdedct.clear()
    tmdrcid.clear()
    tmdred.clear()

In [208]:
# This function is required to access the global scope in case of issues
def tmd_collector(tmit=None, tmsl=['info'], out='file', ofn="tmd_data.txt", cti=5000, dbg=None, startfromid=None, kiea=True):
    if dbg:
        dln = dbg
    else:
        dln = len(tmit)
    outF = open(ofn, "w")
    dbg_ct = 0
    prior = None
    for tmid in tmit:
        if (tmid != prior) or (tmid == startfromid):
            if startfromid:
                startfromid = None
            tmdrcid.clear()
            tmdrcid.append(tmid)
            try:
                if 'info' in tmsl:
                    tdsmqi = tds.Movies(tmid).info()
                    if out=='file':
                        outF.write(str(tdsmqi))
                        outF.write(",")
                    else:
                        tmdtdil.append(tdsmqi)
                    dbg_ct += 1
                    if (dln>=100):
                        if ((dbg_ct % (dln/100)) == 0):
                            print(f'Records Parsed: {dbg_ct} out of {dln}')
            except Exception as error:
                error_string = str(error)
                if error_string in tmdedct:
                    tmdedct[error_string] +=1
                else:
                    tmdedct[error_string] =1
                if error_string == 'KeyboardInterrupt':
                    if kiea == False:
                        print('Continue? y/n')
                        usrinp = input()
                        if usrinp =='n':
                            print('Exiting!')
                            print(f"Forcibly exited collection at record: {tmid}")
                            break
                        else:
                            continue
                    else:
                        break
        if tmid == prior:
            if error_string in edct:
                tmdedct[error_string] +=1
            else:
                tmdedct[error_string] =1
            continue
        if dbg:
            if dbg_ct >= dbg:
                break
    if out=='file':
        outF.close()
    print('API Call Finished!')
    print(f"Finished collection at record: {tmid}")

In [173]:
def makeframe_fromlst(inp, ind=None):
    df = pd.DataFrame(inp)
    if ind:
        df.sort_values(ind, inplace = True)
        df.set_index(ind, inplace = True)
    return df

In [172]:
def makeframe_fromtxt(inp, ind=None):
    df = pd.DataFrame(list(ast.literal_eval(open(inp, 'r').read())))
    if ind:
        df.sort_values(ind, inplace = True)
        df.set_index(ind, inplace = True)
    return df

In [412]:
tmovus

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,ordering,title,region,language,types,attributes,isOriginalTitle,directors,writers,averageRating,numVotes
0,tt0000630,movie,Hamlet,Amleto,0,1908,None,None,Drama,4,Hamlet,US,None,None,None,0,nm0143333,nm0000636,3.8,12
1,tt0000679,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0,1908,None,120,"Adventure,Fantasy",1,The Fairylogue and Radio-Plays,US,None,None,None,0,"nm0877783,nm0091767","nm0000875,nm0877783",5.2,35
2,tt0000886,movie,"Hamlet, Prince of Denmark",Hamlet,0,1910,None,None,Drama,2,"Hamlet, Prince of Denmark",US,None,None,None,0,nm0099901,nm0000636,5.0,24
3,tt0001101,movie,Abraham Lincoln's Clemency,Abraham Lincoln's Clemency,0,1910,None,None,None,2,Abraham Lincoln's Clemency,US,None,None,None,0,nm0923603,None,5.2,13
4,tt0001159,movie,The Connecticut Yankee,The Connecticut Yankee,0,1910,None,None,None,1,The Connecticut Yankee,US,None,None,None,0,None,None,5.6,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142871,tt9913660,movie,No Apology,No Apology,0,2019,None,102,Drama,4,No Apology,US,None,imdbDisplay,None,0,nm1067844,nm3014403,5.4,41
142872,tt9914642,movie,Albatross,Albatross,0,2017,None,97,Documentary,1,Albatross,US,None,None,None,0,nm5300859,"nm5300859,nm7332227",8.2,28
142873,tt9914644,movie,9/11: Escape from the Towers,9/11: Escape from the Towers,0,2018,None,120,Documentary,2,9/11: Escape from the Towers,US,None,imdbDisplay,None,0,nm10537376,None,8.3,67
142874,tt9916190,movie,Safeguard,Safeguard,0,2020,None,90,"Action,Adventure,Thriller",5,Safeguard,US,None,imdbDisplay,None,0,nm7308376,nm7308376,3.4,140


In [229]:
tds.Find('tt0133093').info(external_source='imdb_id')

{'movie_results': [{'poster_path': '/f89U3ADr1oiB1s9GkdPOEpXUk5H.jpg',
   'title': 'The Matrix',
   'video': False,
   'vote_average': 8.1,
   'overview': 'Set in the 22nd century, The Matrix tells the story of a computer hacker who joins a group of underground insurgents fighting the vast and powerful computers who now rule the earth.',
   'release_date': '1999-03-30',
   'adult': False,
   'backdrop_path': '/fNG7i7RqMErkcqhohV2a6cV1Ehy.jpg',
   'id': 603,
   'genre_ids': [28, 878],
   'vote_count': 18912,
   'original_language': 'en',
   'original_title': 'The Matrix',
   'popularity': 51.948}],
 'person_results': [],
 'tv_results': [],
 'tv_episode_results': [],
 'tv_season_results': []}

In [59]:
tds.Movies(603).info()

{'adult': False,
 'backdrop_path': '/fNG7i7RqMErkcqhohV2a6cV1Ehy.jpg',
 'belongs_to_collection': {'id': 2344,
  'name': 'The Matrix Collection',
  'poster_path': '/lh4aGpd3U9rm9B8Oqr6CUgQLtZL.jpg',
  'backdrop_path': '/bRm2DEgUiYciDw3myHuYFInD7la.jpg'},
 'budget': 63000000,
 'genres': [{'id': 28, 'name': 'Action'},
  {'id': 878, 'name': 'Science Fiction'}],
 'homepage': 'http://www.warnerbros.com/matrix',
 'id': 603,
 'imdb_id': 'tt0133093',
 'original_language': 'en',
 'original_title': 'The Matrix',
 'overview': 'Set in the 22nd century, The Matrix tells the story of a computer hacker who joins a group of underground insurgents fighting the vast and powerful computers who now rule the earth.',
 'popularity': 51.948,
 'poster_path': '/f89U3ADr1oiB1s9GkdPOEpXUk5H.jpg',
 'production_companies': [{'id': 79,
   'logo_path': '/tpFpsqbleCzEE2p5EgvUq6ozfCA.png',
   'name': 'Village Roadshow Pictures',
   'origin_country': 'US'},
  {'id': 372,
   'logo_path': None,
   'name': 'Groucho II Film

In [58]:
list(tds.Movies(603).info().keys())

['adult',
 'backdrop_path',
 'belongs_to_collection',
 'budget',
 'genres',
 'homepage',
 'id',
 'imdb_id',
 'original_language',
 'original_title',
 'overview',
 'popularity',
 'poster_path',
 'production_companies',
 'production_countries',
 'release_date',
 'revenue',
 'runtime',
 'spoken_languages',
 'status',
 'tagline',
 'title',
 'video',
 'vote_average',
 'vote_count']

# Pickle Code Examples

# Extracting Data from TMDb via their id archives
##### The order in which data will be pulled from the rate-limited TMDb API using their own IDs.

In [181]:
g_reset_tmd()

In [198]:
# debug file for test models
tmd_collector(tmit=sorted_tmdids, ofn = 'debug.txt', dbg=1000)

Records Parsed: 10 out of 1000
Records Parsed: 20 out of 1000
Records Parsed: 30 out of 1000
Records Parsed: 40 out of 1000
Records Parsed: 50 out of 1000
Records Parsed: 60 out of 1000
Records Parsed: 70 out of 1000
Records Parsed: 80 out of 1000
Records Parsed: 90 out of 1000
Records Parsed: 100 out of 1000
Records Parsed: 110 out of 1000
Records Parsed: 120 out of 1000
Records Parsed: 130 out of 1000
Records Parsed: 140 out of 1000
Records Parsed: 150 out of 1000
Records Parsed: 160 out of 1000
Records Parsed: 170 out of 1000
Records Parsed: 180 out of 1000
Records Parsed: 190 out of 1000
Records Parsed: 200 out of 1000
Records Parsed: 210 out of 1000
Records Parsed: 220 out of 1000
Records Parsed: 230 out of 1000
Records Parsed: 240 out of 1000
Records Parsed: 250 out of 1000
Records Parsed: 260 out of 1000
Records Parsed: 270 out of 1000
Records Parsed: 280 out of 1000
Records Parsed: 290 out of 1000
Records Parsed: 300 out of 1000
Records Parsed: 310 out of 1000
Records Parsed: 3

In [200]:
retrieval_date = datetime.date.today()- datetime.timedelta(days=1)

datetime.date(2021, 4, 2)

In [201]:
datetime.date.strftime(datetime.date.today(), '%m_%d_%Y')

'04_03_2021'

In [ ]:
# The API pulls at a rate of 1 row per second. The full dataset will take a week to pull.
#tmd_collector(tmit=sorted_tmdids, ofn=f"tmd_data_on_{datetime.date.strftime(datetime.date.today(), '%m_%d_%Y')}.txt")

# Global Variable Checkout Section for TMDb Pulls
To be rewritten in final without global variables if feasible.

In [210]:
tmdtdil

[]

In [211]:
tmdedct

{'404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/148002?api_key=f9d608bfe2388abbf6f6c962be3d88fe': 1,
 '404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/166334?api_key=f9d608bfe2388abbf6f6c962be3d88fe': 1,
 '404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/258389?api_key=f9d608bfe2388abbf6f6c962be3d88fe': 1,
 '404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/289495?api_key=f9d608bfe2388abbf6f6c962be3d88fe': 1,
 '404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/294730?api_key=f9d608bfe2388abbf6f6c962be3d88fe': 1,
 '404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/304388?api_key=f9d608bfe2388abbf6f6c962be3d88fe': 1,
 '404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/314487?api_key=f9d608bfe2388abbf6f6c962be3d88fe': 1,
 '404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/340042?api_key=f9d608bfe2388abbf6f6c9

In [212]:
tmdrcid

[539617]

In [213]:
tmdred

{}

In [177]:
makeframe_fromtxt('tmd_data.txt', ind='id')

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
id,,,,,,,,,,,,,,,,,,,,,,,,
2,False,/hQ4pYsIbP22TMXOUdSfC2mjWrO0.jpg,None,0,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",,tt0094675,fi,Ariel,Taisto Kasurinen is a Finnish coal miner whose...,8.974,/ojDg0PGvs6R9xYFodRct2kdI6wC.jpg,"[{'id': 2303, 'logo_path': None, 'name': 'Vill...","[{'iso_3166_1': 'FI', 'name': 'Finland'}]",1988-10-21,0,73,"[{'english_name': 'German', 'iso_639_1': 'de',...",Released,,Ariel,False,6.8,129
3,False,/l94l89eMmFKh7na2a1u5q67VgNx.jpg,None,0,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",,tt0092149,fi,Varjoja paratiisissa,"An episode in the life of Nikander, a garbage ...",7.779,/nj01hspawPof0mJmlgfjuLyJuRN.jpg,"[{'id': 2303, 'logo_path': None, 'name': 'Vill...","[{'iso_3166_1': 'FI', 'name': 'Finland'}]",1986-10-17,0,74,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,,Shadows in Paradise,False,7.2,127
5,False,/u0zMKKpEdDWpOKmFW2sLbKKICJH.jpg,None,4000000,"[{'id': 80, 'name': 'Crime'}, {'id': 35, 'name...",,tt0113101,en,Four Rooms,It's Ted the Bellhop's first night on the job....,13.645,/75aHn1NOYXh4M7L5shoeQ6NGykP.jpg,"[{'id': 14, 'logo_path': '/m6AHu84oZQxvq7n1rsv...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-09,4257354,98,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Twelve outrageous guests. Four scandalous requ...,Four Rooms,False,5.7,1916


# Rewrite the Following Section!
# IMDb data is no longer matched to TMDb data during API calls!

# Seeking Data from TMDb via IMDb
##### The order in which data will be pulled from the rate-limited TMDb API using IMDb IDs.

In [ ]:
g_reset_imt()

In [ ]:
im_to_tm_collector(imdf=tmovus) # when testing use (imdf=tmovus, dbg=25)

# Global Variable Checkout Section for IM+TM Db Pulls
To be rewritten in final without global variables if feasible.

In [ ]:
imttdil

In [ ]:
imtedct

In [ ]:
imtrcid

# End of Pulls

In [158]:
gnrst=set()
for g in imovdf['genres'].unique():
    if g:
        gnrst.update(g.split(','))

In [159]:
gnrst

{'Action',
 'Adult',
 'Adventure',
 'Animation',
 'Biography',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'Film-Noir',
 'Game-Show',
 'History',
 'Horror',
 'Music',
 'Musical',
 'Mystery',
 'News',
 'Reality-TV',
 'Romance',
 'Sci-Fi',
 'Short',
 'Sport',
 'Talk-Show',
 'Thriller',
 'War',
 'Western'}

In [291]:
nbdf

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0072308,tt0031983,tt0053137,tt0050419"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0037382,tt0071877,tt0038355,tt0117057"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,soundtrack,music_department","tt0054452,tt0057345,tt0049189,tt0056404"
3,nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0077975,tt0072562,tt0078723,tt0080455"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0069467,tt0050986,tt0060827,tt0050976"
...,...,...,...,...,...,...
10820660,nm9993714,Romeo del Rosario,\N,\N,"animation_department,art_department",tt2455546
10820661,nm9993716,Essias Loberg,\N,\N,NaN,\N
10820662,nm9993717,Harikrishnan Rajan,\N,\N,cinematographer,tt8736744
10820663,nm9993718,Aayush Nair,\N,\N,cinematographer,\N


In [287]:
tpdf[(tpdf.tconst.isin(tabdf['tconst'])) & (tpdf['category']!='director')& (tpdf['category']!='writer')]

,tconst,ordering,nconst,category,job,characters
849,tt0000502,1,nm0215752,actor,\N,\N
850,tt0000502,2,nm0252720,actor,\N,\N
3709,tt0001028,2,nm0378408,producer,producer,\N
4246,tt0001115,10,nm0630641,actress,\N,"[""Dansemusen""]"
4247,tt0001115,1,nm0064953,actor,\N,"[""Chief of Police""]"
...,...,...,...,...,...,...
43847859,tt9915306,8,nm10537857,actress,\N,"[""Razia Don""]"
43847860,tt9915306,9,nm9583802,actress,\N,\N
43848817,tt9915808,1,nm4667299,actress,\N,"[""Alicia""]"
43848818,tt9915808,2,nm5014622,actor,\N,"[""Paul""]"


In [153]:
tpdf[tpdf['tconst']=='tt0000574']

,tconst,ordering,nconst,category,job,characters
1040,tt0000574,10,nm0675239,cinematographer,director of photography,\N
1041,tt0000574,1,nm0846887,actress,\N,"[""Kate Kelly""]"
1042,tt0000574,2,nm0846894,actor,\N,"[""School Master""]"
1043,tt0000574,3,nm3002376,actor,\N,"[""Steve Hart""]"
1044,tt0000574,4,nm0170118,actress,\N,\N
1045,tt0000574,5,nm0846879,director,\N,\N
1046,tt0000574,6,nm0317210,producer,producer,\N
1047,tt0000574,7,nm0425854,producer,producer,\N
1048,tt0000574,8,nm0846911,producer,producer,\N
1049,tt0000574,9,nm2421834,composer,\N,\N


In [167]:
ckd = {'actress': 'f', 'actor': 'm'}
list(ckd.keys())
tpdf[(tpdf['tconst']=='tt0000574') & tpdf['category'].isin(list(ckd.keys()))]

,tconst,ordering,nconst,category,job,characters
1041,tt0000574,1,nm0846887,actress,\N,"[""Kate Kelly""]"
1042,tt0000574,2,nm0846894,actor,\N,"[""School Master""]"
1043,tt0000574,3,nm3002376,actor,\N,"[""Steve Hart""]"
1044,tt0000574,4,nm0170118,actress,\N,\N


In [29]:
def idparse(x, tp='Many to One', sdf0=None, scol0=None, rcol0=None, sdf1=None, scol1=None, rcol1=None, ecol=None, enc=None):
    nidl = list()
    nml = list()
    if (x): # If not a null char
        if x!='\\N':
            if tp == 'Many to One':
                # Using a cell of many ids in a string on a column with 1 matching row per id
                nidl = x.split(",")
                nml = list()
                for nid in nidl:
                    nml.append(sdf0[sdf0[scol0]==nid][rcol0])
                return ", ".join((str(v) for v in nml))
                # Returns a similar string

            if tp == 'Only One':
                # Use only 1 value to match to 1 matching row
                return sdf0[sdf0[scol0]==x][rcol0]
            if tp == 'One to Many':
                # Use 1 value to collect a string list of matches
                nsdf = sdf1
                nscol = scol1
                nrcol = rcol1
                nidl = list()
                doOnce = 0
                nst = ""
                sc = ""
                for item in sdf0[(sdf0[scol0]==x) & (sdf0[ecol].isin(list(enc.keys())))]:
                        nst += sc + f"{idparse(item, tp='Only One', sdf0 = nsdf, scol0 = nscol, rcol0 = nrcol)} ({enc[sdf0[ecol]]})"
                        if doOnce == 0:
                            sc = ", "
                            doOnce = 1
                return nst

In [30]:
def findnbdfn(x):
    return idparse(x, tp='Many to One', sdf0=nbdf, scol0='nconst', rcol0='primaryName')

In [186]:
tpdf

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,\N,"[""Self""]"
1,tt0000001,2,nm0005690,director,\N,\N
2,tt0000001,3,nm0374658,cinematographer,director of photography,\N
3,tt0000002,1,nm0721526,director,\N,\N
4,tt0000002,2,nm1335271,composer,\N,\N
...,...,...,...,...,...,...
43887525,tt9916880,5,nm0996406,director,principal director,\N
43887526,tt9916880,6,nm1482639,writer,\N,\N
43887527,tt9916880,7,nm2586970,writer,books,\N
43887528,tt9916880,8,nm1594058,producer,producer,\N


In [36]:
nbdf

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0031983,tt0050419,tt0053137,tt0072308"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0117057,tt0037382,tt0038355,tt0071877"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,soundtrack,music_department","tt0057345,tt0056404,tt0054452,tt0049189"
3,nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0072562,tt0078723,tt0077975,tt0080455"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0060827,tt0069467,tt0050986,tt0050976"
...,...,...,...,...,...,...
10825471,nm9993714,Romeo del Rosario,\N,\N,"animation_department,art_department",tt2455546
10825472,nm9993716,Essias Loberg,\N,\N,NaN,\N
10825473,nm9993717,Harikrishnan Rajan,\N,\N,cinematographer,tt8736744
10825474,nm9993718,Aayush Nair,\N,\N,cinematographer,\N


# First pass of data parsing written. Write initial models.

## Features in datasets to aim models towards

### Combined IMDb dataframe at this moment
Immediate: genre, rating, number of votes, year, runtime minutes
Later: directors, writers, actors (with gender and name)

### Incoming TMDb data sourced: unique from IMDb
budget, revenue, popularity, description
duplicates to cross reference: ratings and votes

### TMDb data to pull in the future for analysis
keywords (to use in subgenre analysis

LogisticRegression, RandomForest
Rate from 0 to 10: normal regression, or random forest regressor. Prefer Random Forest

Rating good or bad: classifier
Trying to rate from zero to 5: regressor

Note: I don't know how to cross-validate yet.